## Проектная работа
---

1. [Импорт данных и ознакомление](#start)
2. [Предобработка данных](#step_2)
    * [Восполнение пропусков в данных](#step_2_1)
    * [Приведение к целочисленному типу данных](#step_2_2)
    * [Проверка на явные дубликаты](#step_2_3)
    * [Лемматизация целей кредита](#step_2_4)
    * [Категоризация данных](#step_2_5)
3. [Анализ данных](#step_3)
    * [Зависимость между количеством детей и возвратом кредита в срок](#step_3_1)
    * [Зависимость между семейным положением и возвратом кредита в срок](#step_3_2)
    * [Зависимость между уровнем дохода и возвратом кредита в срок](#step_3_3)
    * [Зависимость между целью кредита и возвратом кредита в срок](#step_3_4)
4. [Выводы](#step_4)

В данном проекте необходимо обределить, как влияет ряд параметров на платежеспособность клиента. Для проведения работы предоставлен файл `data.csv`, в котором приведены данные о клиентах банка.

Спецификация столбцов:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

<a id="start"></a>
### Часть 1. Импорт данных и ознакомление
---

В этой части произведем импорт даннных и первичный анализ фрейма.

In [251]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

data = pd.read_csv('data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [252]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [253]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В ряде столбцов представленны категориальные данные. Проверим их на наличие пропуском, неявных дублей и потенциальных ошибок.

In [254]:
columns_unique = ['children', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt']

for column in columns_unique:
    print(column, data[column].unique())

children [ 1  0  3  2 -1  4 20  5]
education_id [0 1 2 3 4]
family_status_id [0 1 2 3 4]
gender ['F' 'M' 'XNA']
income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
debt [0 1]


In [255]:
display(data[data['children'] == 20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью



По итогу первичного ознакомления с данными отмечаем следующие моменты:

1) В столбце `'days_employed'` имеются отрицательные значения, что невозможно. Скорее всего данная ошибка носит технологичесикй характер. Для её исправления следует получить абсолютные значения по столбцу. Такой прием возможен так как данные не превышают рамочных значений, и явных указаний на то, что им не следует доверять нет.

2) Так же в столбце `'days_employed'` имеются значения, значительно превышающие рамочные. Об их природе сказать сложно, однако такие данные использовать в анализе не следует, ибо под сомнение о правдивости попадает вся строка. Строки с такими ошибками следует исключить из выборки.

3) В столбцах `'days_employed'` и `'total_income'` имеются пропуски. Можно предположить, что пропуски в этих столбцах встречаются одновременно, и вызваны человеческим фактором - если человек не указал свой трудовой стаж, то и не стал указывать свой доход. Для проверки этой гипотезу посмотрим, сколько строк имеют пропуски в обоих столбцах одновременно.

4) В столбце `'children'` есть значение `"-1"` и `"20"`. В первом случае клиент, скорее всего, не предоставил информации, и система проставила данное значение в автоматическом режиме. Во втором случае, следует подвергнуть сомнению, что в выборке из 21525 клиентов, имеются такие, у кого 20 детей, особенно на фоне того, что ближайшее значение равно 5. Природу данной ошибки определить сложно, мог сыграть как человеческий фактор: не хотели предоставлять достоверную информацию, так ошибка могла возникнуть и при преобразовании данных во время выгрузки. Для дальнейшего анализа обеъдиним эти значение.

5) В столбце `'gender'` имеется значение `'XNA'`, которое так же скорее всего проставленно из-за отсутсвии информации от клиента. Это категорийное значение - производить замену не обязательно.

<a id="step_2"></a>
### Часть 2. Предобработка данных.
---
Произведем предобработку данных, для последующего анализа.

<a id="step_2_1"></a>
#### Часть 2.1. Восполнение пропусков в данных.
---

Для восполнения пропусков в данных подробнее ознакомимся с фреймом.

In [256]:
data[(data['days_employed'].isna()) & data['total_income'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Гипотеза о том, что если в строке есть пропуск в одном столбце, то есть и в другом подствердилась. Для принятия решения, о способе заполнения пропусков, получим таблицу без пропусков, а так же с абсолютными значениями по столбцу `'days_employed'`, после чего найдем среднее значение, медиану, максимум и минимум для ряда столбцов.

In [257]:
data_not_NaN = data.dropna(subset=['days_employed']).reset_index(drop=True)
data_not_NaN['days_employed'] = data_not_NaN['days_employed'].abs()
data_not_NaN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


In [258]:
columns_chek = ['days_employed', 'dob_years', 'total_income']
data_ch = []

# получим средние, медианные, максимальные и минимальные значения по списку столбцов columns_chek
for column in columns_chek:
    data_md = []
    data_md.append(data_not_NaN[column].mean())
    data_md.append(data_not_NaN[column].median())
    data_md.append(data_not_NaN[column].max())
    data_md.append(data_not_NaN[column].min())
    data_ch.append(data_md)

data_ch_f = pd.DataFrame(data=data_ch, columns=['mean','median', 'max', 'min'], index=columns_chek).astype('int64')
display(data_ch_f)

,mean,median,max,min
days_employed,66914,2194,401755,24
dob_years,43,42,75,0
total_income,167422,145017,2265604,20667


Из полученной таблицы видим:
1) В столбце `'days_employed'` имеются строки, со значениями сильно превышающими, возможные. Если взять за основу, что человек, может оффициально работать с **16 лет**, то для максимального возраста в выборке, получим максимальный стаж в днях **меньше 21594** `(из расчета (75 - 16) х 366)`. Необходимо проверить всю выборку.

2) В столбце `'dob_years'` имеются строки с значением **0**. Получение кредита, а значит и наличие кредитной истории возможны **с 18 лет**. Необходимо проверить всю выборку на это условие.

Причина возникновения данных ошибок неизвестна.

In [259]:
print('Строк со стажем, больше максимально возможного: ',
      data_not_NaN[data_not_NaN['days_employed'] > ((data_not_NaN['dob_years'] - 16) * 366)]['children'].count())
print('Строк с возрастом, меньше возможного: ',
      data_not_NaN[data_not_NaN['dob_years'] < 18]['children'].count())

Строк со стажем, больше максимально возможного:  3608
Строк с возрастом, меньше возможного:  91


По выборке посчитаем медианную зарплату и стаж, заполним медианными значениями пропуски в исходной таблице в столбце `'total_income'` и `'days_employed'`.

In [281]:
# получаем медианы зарплаты и стажа
median_total_income = data_not_NaN['total_income'].median()
median_dayes_employed = data_not_NaN['days_employed'].median()

# копируем исходный фрейм, заменяем пропуски и неверные данные
data_prefinal = data.copy(deep=True)
data_prefinal['total_income'] = data_prefinal['total_income'].fillna(median_total_income)
data_prefinal['days_employed'] = data_prefinal['days_employed'].fillna(median_dayes_employed)
data_prefinal['days_employed'] = data_prefinal['days_employed'].abs()
data_prefinal.loc[data_clear['children'] == 20, 'children'] = -1

data_prefinal.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [282]:
print(f'Строк в новой выборке: {data_prefinal.shape[0]}\nСтрок с пропусками:\n{data_prefinal.isna().sum()}')

Строк в новой выборке: 21525
Строк с пропусками:
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**По итогу первой части предварительной обработки:**

1) Были найдены строки с пропусками в данных. Пропуски являются следствием человеческого фактора: скорее всего люди просто не предоставили данные о своем трудовом стаже и ежемесячной зарплате.

2) Было обнаружено, что в данных имеются строки с значениями, выходящими за пограничные: стаж больше чем: `(возраст - возраст возможного устройства на работу) * 366 дней`; возраст человека предоставившего данные о себе `меньше 18 лет`. Данные строки были удалены из итоговой выборки. Так как в них ставится поод сомнение данных из других столбцов.

3) Была расчитана медианная зарплата по выборке без пропусков в данных и строк со значениями вне граничных.

4) Пропуски в столбце `'total_income'` в итоговой выборке были заполнены медианной зарплатой из п.3, а пропуски из столбца `'days_emploued'` заполнены максимально возможным стажем (формула из п.2).

<a id="step_2_2"></a>
#### Часть 2.2. Приведение к целочисленному типу данных.


Для дальнейшой предварительной обработки заменим вещественный тип данных в столбцах `'days_employed'`, `'total_income'` на целочисленный. Потерянная точность не должна повлиять на решение поставленной задачи.

In [263]:
columns_type_replace = ['days_employed', 'total_income']

for column in columns_type_replace:
    data_prefinal[column] = data_prefinal[column].astype('int64')
data_prefinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Перевод вещественных значений в целочисленные выполнен. Применялся метод `astype()`, так как альтернатива в виде метода `to_numeric()` не подходит, ввиду того, что этот метод переводит данные только в тип float.

<a id="step_2_3"></a>
#### Часть 2.3. Проверка на явные дубликаты.
---

Воспользуемся методом `.duplicated()` для поиска дубликатов в выборке `data_prefinal`.

In [264]:
print('Количество явных дубликатов:', data_prefinal.duplicated().sum())

Количество явных дубликатов: 54


In [265]:
data_prefinal = data_prefinal.drop_duplicates().reset_index(drop=True)
print('Проверка, количество дубликатов:', data_prefinal.duplicated().sum())

Проверка, количество дубликатов: 0


Было обнаружено 54 дубликата, для их удаления использовался метод `drop_duplicates()`, для пересчета индексов метод `reset_index(drop=True)`

<a id="step_2_4"></a>
#### Часть 2.4. Лемматизация целей кредита.
---

Для дальнейшего анализа, необходимо привести цели, под которые запрашивали кредит клиенты банка к нескольким  категориям. Для этого воспользуемся библиотекой `pymystem3` и создадим словать вида: `"целевая категория" : ["входная цель_1", "входная цель_2" и т.д.]`

In [266]:
m = Mystem()
value = list(data_prefinal['purpose'].unique())
# получаем все уникальные цели кредита
lemma_list = []
dict_lemma = dict()
for val in value:
    lemmas = m.lemmatize(val)
    lem = ' '.join(lemmas)
    dict_lemma[lem] = val
# формируем первичный словарь лемма - исходное значение из фрейма.
    lemma_list.append(lem)
    
# создаем чистый итоговый словарь
clear_dict_purpose = dict()

for lemma in dict_lemma:
# проходим первичный словарь и по ключевым словам распределяем цели по категориям
    if 'жилье' in lemma or 'недвижимость' in lemma:
        if 'недвижимость' not in clear_dict_purpose:
            clear_dict_purpose['недвижимость'] = list()
            clear_dict_purpose['недвижимость'].append(dict_lemma[lemma])
        else:
            clear_dict_purpose['недвижимость'].append(dict_lemma[lemma])
    elif 'образование' in lemma:
        if 'образование' not in clear_dict_purpose:
            clear_dict_purpose['образование'] = list()
            clear_dict_purpose['образование'].append(dict_lemma[lemma])
        else:
            clear_dict_purpose['образование'].append(dict_lemma[lemma])
    elif 'автомобиль' in lemma:
        if 'автомобиль' not in clear_dict_purpose:
            clear_dict_purpose['автомобиль'] = list()
            clear_dict_purpose['автомобиль'].append(dict_lemma[lemma])
        else:
            clear_dict_purpose['автомобиль'].append(dict_lemma[lemma])
    elif 'свадьба' in lemma:
        if 'свадьба' not in clear_dict_purpose:
            clear_dict_purpose['свадьба'] = list()
            clear_dict_purpose['свадьба'].append(dict_lemma[lemma])
        else:
            clear_dict_purpose['свадьба'].append(dict_lemma[lemma])

# выводим получившийся словать категорий
for sl in clear_dict_purpose:
        print(sl, clear_dict_purpose[sl])

недвижимость ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью', 'покупка своего жилья', 'операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилью']
автомобиль ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобиль', 'сделка с подержанным автомобилем', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']
образование ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования', 'получение дополнительного образования', 'получение высшего образования', 'профильное образование', 'высшее образование', 'заняться высшим образованием']
свадьба ['сыграть свадьбу', 'на прове


Мы получили словарь категорий, который будет необходим при ответе на вопросы задания.

<a id="step_2_5"></a>
#### Часть 2.5. Категоризация данных
---

Для начала используем словарь категорий целей кредита и созданим новый столбец в исходных данных с помощью метода `apply()` и функции `rename`.

In [267]:
def rename(value):
# создадим функцию, которая принимает значения из столбца purpose
# сверяет эти значение с значениями словаря из прошлой части работы
# и возвращает  ключи - категорию цели кредита
    for sl in clear_dict_purpose:
        if value in clear_dict_purpose[sl]:
            return sl

data_prefinal['purpose_type'] = data_prefinal['purpose'].apply(rename)
data_prefinal.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


Далее, произведем категоризацию по уровню дохода и возрасту. ДЛя этого создадим две функции: `total_income_remane` `age_rename`, которые будут получать значение из соответствующего столба и путем фильтрации присваивать новую категорию.

По уровню дохода возьмем шаг 50000 у.е. По возрасту, разделим всю выборку на три группы 18-34, 35-64, >=65.

In [268]:
def total_income_remane(value):
# функция категоризации по уровню дохода
    if value >= 0 and value < 100000:
        return 'до 100к у.е'
    elif value >= 100000 and value < 135000:
        return 'до 135к у.е'
    elif value >= 135000 and value < 155000:
        return 'до 155к у.е.'
    elif value >= 155000 and value < 210000:
        return 'до 210к у.е.'
    elif value >= 210000:
        return '> 210к у.е.'
    else:
        return 'error'

#применим созданную функцию с соответсвтующему столбцу    
data_prefinal['total_income_cat'] =  data_prefinal['total_income'].apply(total_income_remane)
print(data_prefinal['total_income_cat'].value_counts())


def age_rename(value):
# функция категоризации по возрасту
    if value >= 0 and value < 32:
        return 'до 32'
    elif value >= 32 and value < 40:
        return 'до 40'
    elif value >= 40 and value < 48:
        return 'до 48'
    elif value >= 48 and value < 56:
        return 'до 56'
    elif value >= 56:
        return '> 56'
    else:
        return 'error'

#применим созданную функцию с соответсвтующему столбцу    
data_prefinal['dob_years_cat'] =  data_prefinal['dob_years'].apply(age_rename)
print(data_prefinal['dob_years_cat'].value_counts())

def children_rename(value):
# функция категоризации по возрасту
    if value == 0:
        return 'нет детей'
    elif value > 0:
        return 'есть дети'
    else:
        return 'unknow'

#применим созданную функцию с соответсвтующему столбцу    
data_prefinal['children_cat'] =  data_prefinal['children'].apply(children_rename)
print(data_prefinal['children_cat'].value_counts())

> 210к у.е.     4502
до 100к у.е     4463
до 155к у.е.    4217
до 210к у.е.    4214
до 135к у.е     4075
Name: total_income_cat, dtype: int64
до 40    4566
до 32    4379
> 56     4345
до 48    4313
до 56    3868
Name: dob_years_cat, dtype: int64
нет детей    14107
есть дети     7317
unknow          47
Name: children_cat, dtype: int64


Клиенты были разделены на категории по возрасту и доходу. Рамки категорий выбирались таким образом, чтобы получить максимально равное по численности клиентов категории.

Так же клиенты были разделены на три категории по количеству детей (есть/нет детией и отсутсвтие данных).

Получившаяся таблица неудобна для работы. Созданим новый фрейм, в котором оставим только числовые значения и категории.

In [269]:
data_final = data_prefinal[['children_cat', 'education_id',
                            'family_status_id', 'debt', 'purpose_type', 'total_income_cat', 'dob_years_cat']]
data_final.head(10)

,children_cat,education_id,family_status_id,debt,purpose_type,total_income_cat,dob_years_cat
0,есть дети,0,0,0,недвижимость,> 210к у.е.,до 48
1,есть дети,1,0,0,автомобиль,до 135к у.е,до 40
2,нет детей,1,0,0,недвижимость,до 155к у.е.,до 40
3,есть дети,1,0,0,образование,> 210к у.е.,до 40
4,нет детей,1,1,0,свадьба,до 210к у.е.,до 56
5,нет детей,0,1,0,недвижимость,> 210к у.е.,до 32
6,нет детей,0,0,0,недвижимость,> 210к у.е.,до 48
7,нет детей,1,0,0,образование,до 155к у.е.,до 56
8,есть дети,0,1,0,свадьба,до 100к у.е,до 40
9,нет детей,1,0,0,недвижимость,до 155к у.е.,до 48


Создадим словари категорий, которые в дальнейшем будем использовать.

In [270]:
dict_education = data_prefinal[['education', 'education_id']]
dict_education['education'] = dict_education['education'].str.lower()
dict_education = dict_education.drop_duplicates().reset_index(drop=True)

display(dict_education.sort_values(by='education_id', ascending=True))

dict_family = data_prefinal[['family_status', 'family_status_id']]
dict_family['family_status'] = dict_family['family_status'].str.lower()
dict_family = dict_family.drop_duplicates().reset_index(drop=True)

display(dict_family.sort_values(by='family_status_id', ascending=True))


C:\Users\taywa\anaconda3\envs\praktikum_env_win_new\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


C:\Users\taywa\anaconda3\envs\praktikum_env_win_new\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


<a id="step_3"></a>
### Часть 3. Анализ данных
---
В этой части работы проведем анализ подготовленных нами данных и ответим на вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Для более объективной оценки, и ввиду того, что факторы (наличие детей, семейное положение и т.д.) не являются изолированными - оказывают комплексное воздействие на вероятность возращения кредита в срок, помимо их изолированного рассмотрения, выберем дополнительный параметр - возраст, и будем так же рассматривать компанацию каждого фактора с выбранным.

Это позволит ответить наследующий вопрос - влияют ли факторы из задания сильнее, чем выбранный дополнительный фактор (возраст).

<a id="step_3_1"></a>
#### Часть 3.1. Зависимость между наличием детей и возвратом кредита в срок
---

Создадим сводную таблицу с группировкой по столбцу `debt`, в качестве столбов выберем количество значения параметра `children`. Добавим третий строкой отношение клиентов с просрочками по кредиту к клиентом без просрочек. 

Для итогового вывода создадим словарь, ключи которого - параметр, относительно которого проводим проверку, значение - разница максимального и минимального процента клиентов с просрочками.

In [271]:
data_children = data_final.pivot_table(index='children_cat', columns='debt', values='family_status_id', aggfunc='count')
data_children.loc[:,'%'] = data_children.loc[:,1] / data_children.loc[:,0] * 100
data_children = data_children.sort_values(by='%', ascending=False)

display(data_children.round(2))

data_children.drop(['unknow'], axis=0, inplace=True)
difference_value = list()
difference_value.append(data_children['%'].max() - data_children['%'].min())

debt,0,1,%
children_cat,,,
есть дети,6640,677,10.20
нет детей,13044,1063,8.15
unknow,46,1,2.17


Так же создадим сводную таблицу, группировку произведем по столбцу `debt`, в столбцами будут количество детей и возраст. Так же посчитаем процентное соотношение клиентов с просрочками и без.

In [272]:
data_children = data_final.pivot_table(columns='debt', index=['children_cat', 'dob_years_cat'], values='family_status_id', aggfunc='count')
data_children.loc[:,'%'] = data_children.loc[:,1] / data_children.loc[:,0] * 100
data_children = data_children.sort_values(by='%', ascending=False)
display(data_children.round(2))

,debt,0,1,%
children_cat,dob_years_cat,,,
есть дети,до 32,1715.0,218.0,12.71
нет детей,до 32,2169.0,269.0,12.40
есть дети,до 40,2427.0,251.0,10.34
нет детей,до 40,1706.0,170.0,9.96
unknow,до 40,11.0,1.0,9.09
есть дети,до 48,1606.0,140.0,8.72
нет детей,до 48,2356.0,199.0,8.45
есть дети,до 56,587.0,49.0,8.35
нет детей,до 56,3020.0,204.0,6.75


Из полученной таблицы видно, что разброс доли клиентов с просрочками по платежам заметно выше внутри категории `cgildren`, при разбивке по возрастам. Может выдвинуть предположение - возраст клиента сильнее влияет на отсутствие просрочек по кредиту, чем количество детей у него. Подтверждением гипотезы служит то, что клиенты возрастной группы "молодежь" в 3 из 4 группах (по количеству детей), где имеются просрочки, имеют наивысший их процент.

<a id="step_3_2"></a>
#### Часть 3.2. Зависимость между семейным положением и возвратом кредита в срок.
---

Повторим действия из пункта 3.1. Создадим сводную таблицу, в качестве столбов выберем `family_status_id` в одном случае, `family_status_id` и `dob_years_cat` в другом.

In [273]:
data_family = data_final.pivot_table(columns='debt', index='family_status_id', values='children_cat', aggfunc='count')
data_family.loc[:,'%'] = data_family.loc[:,1] / data_family.loc[:,0] * 100
data_family = data_family.sort_values(by='%', ascending=False)
display(data_family.round(2))
display(dict_family.sort_values(by='family_status_id', ascending=True))

difference_value.append(data_family['%'].max() - data_family['%'].min())

debt,0,1,%
family_status_id,,,
4,2536,274,10.80
1,3775,388,10.28
0,11413,931,8.16
3,1110,85,7.66
2,896,63,7.03


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [274]:
data_family = data_final.pivot_table(columns='debt', index=['family_status_id', 'dob_years_cat'], values='children_cat', aggfunc='count')
data_family.loc[:,'%'] = data_family.loc[:,1] / data_family.loc[:,0] * 100
data_family = data_family.sort_values(by='%', ascending=False)
display(data_family.round(2))
display(dict_family.sort_values(by='family_status_id', ascending=True))

,debt,0,1,%
family_status_id,dob_years_cat,,,
2,до 32,12,2,16.67
3,до 32,134,20,14.93
4,до 32,994,131,13.18
1,до 32,811,105,12.95
4,до 40,452,57,12.61
1,до 40,853,104,12.19
0,до 32,1941,229,11.80
4,до 48,386,42,10.88
1,до 48,809,78,9.64


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Из полученных таблиц видно, что семейное положение даже изолированное имеет большой разброс в доля клиентов с просрочками. При рассмотрении двух критериев одновременно, разброс только увеличивается. Можно предположить, что выбранные факторы, оказывают существенное влияние, на вероятность возвращения кредита в срок.

Изолированно, видно, что люди, не состоящие или же не состоявшие в браке, заметно чаще имеют просрочки. Можно предположить, что причины этому: не приученность к ответственнсти (брак к этому наоборот - располагает), а так же отсутсвтие "страхующего" человека рядом.

<a id="step_3_3"></a>
#### Часть 3.3. Зависимость между уровнем дохода и возвратом кредита в срок.
---

Создадим сводную таблицу, строками будут критерий `debt`, столбами `total_income_cat` и `total_income_cat` + `dob_years_cat`.

In [275]:
data_total_income = data_final.pivot_table(columns='debt', index='total_income_cat', values='children_cat', aggfunc='count')
data_total_income.loc[:,'%'] = data_total_income.loc[:,1] / data_total_income.loc[:,0] * 100
data_total_income = data_total_income.sort_values(by='%', ascending=False)
display(data_total_income.round(2))

difference_value.append(data_total_income['%'].max() - data_total_income['%'].min())


debt,0,1,%
total_income_cat,,,
до 210к у.е.,3844,370,9.63
до 155к у.е.,3856,361,9.36
до 135к у.е,3734,341,9.13
до 100к у.е,4109,354,8.62
> 210к у.е.,4187,315,7.52


In [276]:
data_total_income = data_final.pivot_table(columns='debt', index=['total_income_cat','dob_years_cat' ], values='children_cat', aggfunc='count')
data_total_income.loc[:,'%'] = data_total_income.loc[:,1] / data_total_income.loc[:,0] * 100
data_total_income = data_total_income.sort_values(by='%', ascending=False)
display(data_total_income.round(2))

,debt,0,1,%
total_income_cat,dob_years_cat,,,
до 135к у.е,до 32,759,114,15.02
до 155к у.е.,до 32,785,102,12.99
до 210к у.е.,до 32,793,100,12.61
до 100к у.е,до 32,791,97,12.26
до 210к у.е.,до 40,869,98,11.28
до 135к у.е,до 40,738,82,11.11
до 100к у.е,до 40,710,78,10.99
до 155к у.е.,до 40,797,86,10.79
до 100к у.е,до 48,662,69,10.42


Сильного разброса по долям клиентов с просрочками не наблюдается, при изолированном рассмотрении параметра. Это можно объяснить как тем, что люди адекватно оценивают свои возможности и берут кредиты по силам, там и тем, что банки со своей стороны делают примерно тоже самое - не выдают людям суммы, которые они со своего дохода не смогут вернуть.

При расммотрении двух параметров одновременно, видна тенденция, что возраст, независимо от категории дохода, оказывает более сильное влияние на вероятность возврата кредита без просрочек. Так возрастная группа "молодежь" значительно чаще имеет просрочки, чем любая другая возрастная группа, независимо от уровня дохода. Исключение - пожилые люди со средним уровнем дохода.

<a id="step_3_4"></a>
#### Часть 3.4. Зависимость между целью кредита и возвратом кредита в срок.
---

Создадим сводную таблицу. Строками будет параметр `debt`, столбцами `purpose_type` и `purpose_type` + `dob_years_cat`.

In [277]:
data_purpose_type = data_final.pivot_table(columns='debt', index='purpose_type', values='children_cat', aggfunc='count')
data_purpose_type.loc[:,'%'] = data_purpose_type.loc[:,1] / data_purpose_type.loc[:,0] * 100
data_purpose_type = data_purpose_type.sort_values(by='%', ascending=False)
display(data_purpose_type.round(2))

difference_value.append(data_purpose_type['%'].max() - data_purpose_type['%'].min())

debt,0,1,%
purpose_type,,,
автомобиль,3471,359,10.34
образование,3644,370,10.15
свадьба,2149,186,8.66
недвижимость,10032,782,7.80


In [278]:
data_purpose_type = data_final.pivot_table(columns='debt', index=['purpose_type', 'dob_years_cat'], values='children_cat', aggfunc='count')
data_purpose_type.loc[:,'%'] = data_purpose_type.loc[:,1] / data_purpose_type.loc[:,0] * 100
data_purpose_type = data_purpose_type.sort_values(by='%', ascending=False)
display(data_purpose_type.round(2))

debt                           0    1      %
purpose_type dob_years_cat                  
образование  до 32           731  111  15.18
автомобиль   до 40           720  105  14.58
             до 32           649   93  14.33
свадьба      до 32           423   50  11.82
образование  до 40           743   84  11.31
недвижимость до 32          2005  220  10.97
образование  до 48           730   71   9.73
свадьба      до 40           465   44   9.46
автомобиль   до 48           730   67   9.18
свадьба      до 48           429   38   8.86
недвижимость до 40          2124  185   8.71
образование  до 56           638   48   7.52
недвижимость до 48          2009  150   7.47
автомобиль   до 56           631   45   7.13
свадьба      до 56           385   27   7.01
образование  > 56            802   56   6.98
недвижимость до 56          1877  126   6.71
автомобиль   > 56            741   49   6.61
свадьба      > 56            447   27   6.04
недвижимость > 56           2017  101   5.01

Разброс долей просрочек, при изолированном расмотрении целей кредиты, минимален. Существенного влияния данный параметр на возврат кредита в срок не оказывает.

При расмотрении двух параметров одновременно, видно, что разброс долей просрочек заметно увеличивается, эту пару параметров стоит использовать при оценке платежеспособности клиента, так же как и пару "семенойе положение" и "возраст".

<a id="step_4"></a>
### Часть 4. Выводы.
---

Для написание выводов, рассмотрим, как изолированно возраст влияет на вероятность возврата кредита в срок. Создадим соответствующую сводную таблицу, после чего обратимся к словарю `difference_value` и посмотрим максимальные расбросы долей просрочеч изолированных 4-х параметров из задания и одного дополнительного.

In [279]:
data_age = data_final.pivot_table(columns='debt', index='dob_years_cat', values='children_cat', aggfunc='count')
data_age.loc[:,'%'] = data_age.loc[:,1] / data_age.loc[:,0] * 100
data_age = data_age.sort_values(by='%', ascending=False)
display(data_age.round(2))

difference_value.append(data_age['%'].max() - data_age['%'].min())

debt,0,1,%
dob_years_cat,,,
до 32,3892,487,12.51
до 40,4144,422,10.18
до 48,3974,339,8.53
до 56,3615,253,7.00
> 56,4105,240,5.85


In [280]:
share = pd.Series(data=difference_value, index=['children', 'family', 'total_income', 'purpose', 'age'])
display(share.sort_values(ascending=False).round(2))

age             6.67
family          3.77
purpose         2.55
total_income    2.10
children        2.05
dtype: float64

**Вывод:** из полученных данных видно, что наибольшее влияние на долю просрочек по кредиту, изолированно, оказывает семейное положение кредитующегося (критерий задания) и его возраст (дополнительный критерий, выбран самостоятельно). 

Для оценки вероятности возврата кредита без просрочек следует использовать комбинации параметров, и избегать оценок по изолированным параметрам.